In [1]:
import riiideducation

import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import psutil

import gc
from collections import defaultdict

from tqdm.notebook import tqdm
import psutil
import joblib
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
%%time

feld_needed = ['timestamp','user_id','content_id', 'task_container_id','content_type_id', 'answered_correctly', 'prior_question_elapsed_time', 'prior_question_had_explanation']
train = pd.read_pickle('../input/riiid-answer-correctness-prediction-files/train.pkl')
train = train[feld_needed]
question = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv')

CPU times: user 2.69 s, sys: 4.58 s, total: 7.27 s
Wall time: 32.7 s


In [3]:
question_cmnts = pd.read_csv('../input/mydata/archive/archive/question_cmnts.csv')
_ = gc.collect()

In [4]:
train = train.loc[train.content_type_id == False].reset_index(drop=True)

In [5]:
user_time_last = train[['user_id','timestamp']].groupby('user_id')['timestamp'].last().reset_index()
user_time_last.columns = ['user_id','last_time']
time_last_dict = defaultdict(int)
for cnt,item in enumerate(tqdm(user_time_last[['user_id','last_time']].values)):
    time_last_dict[item[0]] = item[1]/(1000*3600)

In [6]:
train['lag_time'] = train[['user_id','timestamp']].groupby('user_id')['timestamp'].shift()

In [7]:
train['lag_time2'] = train[['user_id','timestamp']].groupby('user_id')['timestamp'].shift(2)

In [8]:
time_mean = 0.009687222222222222
time_mean2 = 0.025640833333333335
time_mean3 = 0.043081666666666664

In [9]:
user_time_last = train[['user_id','lag_time']].groupby('user_id')['lag_time'].max().reset_index()

In [10]:
user_time_last['lag_time'].fillna(-1,inplace=True)

In [11]:
user_time_last.columns = ['user_id','last_time']
time_last_dict2 = defaultdict(int)
for cnt,item in enumerate(tqdm(user_time_last[['user_id','last_time']].values)):
    time_last_dict2[item[0]] = item[1]/(1000*3600)

In [12]:
user_time_last = train[['user_id','lag_time2']].groupby('user_id')['lag_time2'].max().reset_index()

In [13]:
user_time_last['lag_time2'].fillna(-1,inplace=True)

In [14]:
user_time_last.columns = ['user_id','last_time']
time_last_dict3 = defaultdict(int)
for cnt,item in enumerate(tqdm(user_time_last[['user_id','last_time']].values)):
    time_last_dict3[item[0]] = item[1]/(1000*3600)

In [15]:
del train['lag_time']
del train['lag_time2']
_ = gc.collect()

In [16]:
prior_question_elapsed_time_mean = train.prior_question_elapsed_time.dropna().values.mean()
train['prior_question_elapsed_time_mean'] = train['prior_question_elapsed_time'].fillna(prior_question_elapsed_time_mean)
user_time_last = train[['user_id','prior_question_elapsed_time_mean']].groupby('user_id')['prior_question_elapsed_time_mean'].last().reset_index()
user_time_last.columns = ['user_id','prior_lag_time']
prior_time_last_dict = defaultdict(int)
for cnt,item in enumerate(tqdm(user_time_last[['user_id','prior_lag_time']].values)):
    prior_time_last_dict[item[0]] = item[1]/(1000*3600)

In [17]:
train['prior_question_had_explanation'].fillna(False,inplace=True)
explanation_agg = train.groupby('user_id')['prior_question_had_explanation'].agg(['sum', 'count'])

In [18]:
explanation_sum_dict = explanation_agg['sum'].astype('int16').to_dict(defaultdict(int))
explanation_count_dict = explanation_agg['count'].astype('int16').to_dict(defaultdict(int))

In [19]:
del explanation_agg
_ = gc.collect()

In [20]:
attempt_group_by = joblib.load('/kaggle/input/sakt-models/attempt_group_by.pkl.lzma')
train["attempt"] = attempt_group_by

In [21]:
attempt_agg_dict = joblib.load('/kaggle/input/sakt-models/attempt_agg_dict.pkl.lzma')

_ = gc.collect()

In [22]:
def get_max_attempt(user_id,content_id):
    k = (user_id,content_id)

    if k in attempt_agg_dict.keys():
        attempt_agg_dict[k]+=1
        return attempt_agg_dict[k]

    attempt_agg_dict[k] = 1
    return attempt_agg_dict[k]

In [23]:
target = 'answered_correctly'

In [24]:
user_agg = train.groupby('user_id')[target].agg(['sum', 'count'])

In [25]:
user_sum_dict = user_agg['sum'].astype('int16').to_dict(defaultdict(int))
user_count_dict = user_agg['count'].astype('int16').to_dict(defaultdict(int))
del user_agg
_ = gc.collect()

In [26]:
input_path = '../input/mydata/archive/archive/'

In [27]:
answered_correctly_avg_c = pd.read_csv(input_path + 'answered_correctly_avg_c.csv')
answered_correctly_std_c = pd.read_csv(input_path + 'answered_correctly_std_c.csv')
answered_correctly_sum_c = pd.read_csv(input_path + 'answered_correctly_sum_c.csv')
answered_correctly_sum_m = pd.read_csv(input_path + 'answered_correctly_sum_m.csv')
answered_correctly_avg_p = pd.read_csv(input_path + 'answered_correctly_avg_p.csv')
answered_correctly_sum_p = pd.read_csv(input_path + 'answered_correctly_sum_p.csv')
answered_correctly_avg_b = pd.read_csv(input_path + 'answered_correctly_avg_b.csv')
answered_correctly_avg_t = pd.read_csv(input_path + 'answered_correctly_avg_t.csv')
answered_correctly_sum_t = pd.read_csv(input_path + 'answered_correctly_sum_t.csv')

lag_time_avg_b = pd.read_csv(input_path + 'lag_time_avg_b.csv')
lag_time_avg_c = pd.read_csv(input_path + 'lag_time_avg_c.csv')
lag_time_avg_g = pd.read_csv(input_path + 'lag_time_avg_g.csv')
lag_time_avg_m = pd.read_csv(input_path + 'lag_time_avg_m.csv')
lag_time_avg_p = pd.read_csv(input_path + 'lag_time_avg_p.csv')
lag_time_avg_t = pd.read_csv(input_path + 'lag_time_avg_t.csv')

hard_question_sum = pd.read_csv(input_path + 'hard_question_sum.csv')

middle_question_sum = pd.read_csv(input_path + 'middle_question_sum.csv')

easy_question_sum = pd.read_csv(input_path + 'easy_question_sum.csv')
answered_correctly_avg_g = pd.read_csv(input_path + 'answered_correctly_avg_g.csv')
answered_correctly_sum_g = pd.read_csv(input_path + 'answered_correctly_sum_g.csv')
question =pd.read_csv('../input/mydata/archive/archive/questions_df.csv')

In [28]:
question.drop('Unnamed: 0',axis=1,inplace=True)

In [29]:
def add_user_feats_without_update(df,user_sum_dict,user_count_dict):
    acsu = np.zeros(len(df), dtype=np.int32)
    cu = np.zeros(len(df), dtype=np.int32)
    for cnt,row in enumerate(df[['user_id']].values):
        acsu[cnt] = user_sum_dict[row[0]]
        cu[cnt] = user_count_dict[row[0]]

    user_feats_df = pd.DataFrame({'user_correct_cumsum':acsu, 'user_correct_cumcount':cu})
    user_feats_df['user_correctness'] = user_feats_df['user_correct_cumsum'] / user_feats_df['user_correct_cumcount']
    df = pd.concat([df, user_feats_df], axis=1)
    return df


def add_explanation(df,explanation_sum_dict,explanation_count_dict):
    acsu_explanation = np.zeros(len(df), dtype=np.int32)
    cu_explanation = np.zeros(len(df), dtype=np.int32)
    for cnt,row in enumerate(df[['user_id','prior_question_had_explanation']].values):
        cu_explanation[cnt] = explanation_count_dict[row[0]] + 1
        acsu_explanation[cnt] = explanation_sum_dict[row[0]] + row[1]
    explanation_feats_df = pd.DataFrame({'explanation_cumsum':acsu_explanation, 'explanation_cumcount':cu_explanation})
    explanation_feats_df['explanation_mean'] = explanation_feats_df['explanation_cumsum'] / explanation_feats_df['explanation_cumcount']
    df = pd.concat([df,explanation_feats_df], axis=1)
    return df


def add_time(df, time_last_dict,time_last_dict2,time_last_dict3):
    time = np.zeros(len(df), dtype=np.float64)
    time2 = np.zeros(len(df), dtype=np.float64)
    time3 = np.zeros(len(df), dtype=np.float64)
    for cnt,row in enumerate(df[['user_id','timestamp']].values):
        if row[0] in time_last_dict.keys():
            time[cnt]=row[1]-time_last_dict[row[0]]
            if(time_last_dict2[row[0]]==time_mean2 or time_last_dict2[row[0]] == -1):#
                time2[cnt]=time_mean2
                time3[cnt]=time_mean3
               
            else:
                time2[cnt]=row[1]-time_last_dict2[row[0]]
                if(time_last_dict3[row[0]]==time_mean3 or time_last_dict3[row[0]] == -1):
                    time3[cnt]=time_mean3 
                else:
                    time3[cnt]=row[1]-time_last_dict3[row[0]]

                time_last_dict3[row[0]]=time_last_dict2[row[0]]

            time_last_dict2[row[0]]=time_last_dict[row[0]]
            time_last_dict[row[0]]=row[1]

        else:
            time[cnt]=time_mean
            time_last_dict[row[0]] = row[1]
            time2[cnt]=time_mean2
            time_last_dict2[row[0]] = time_mean2
            time3[cnt]=time_mean3
            time_last_dict3[row[0]] = time_mean3

    
    time_last_df = pd.DataFrame({'lag_time':time,'lag_time2':time2,'lag_time3':time3})
    
    df = pd.concat([df, time_last_df], axis=1)
    return df


def add_prior_time(df,prior_time_last_dict):
    prior_time = np.zeros(len(df), dtype=np.float64)
    for cnt,row in enumerate(df[['user_id','prior_question_elapsed_time_mean']].values):
        if prior_time_last_dict[row[0]] != 0:
            prior_time[cnt] = row[1] - prior_time_last_dict[row[0]]
            prior_time_last_dict[row[0]] = row[1]
        else:
            prior_time_last_dict[row[0]] = row[1]
            
    time_last_df = pd.DataFrame({'prior_lag_time':prior_time})
    df = pd.concat([df, time_last_df], axis=1)
    return df

In [30]:
def update_user_feats(df, user_sum_dict,user_count_dict):
    temp = []
    for row in df[['user_id','answered_correctly','content_type_id']].values:
        if row[2] == 0:
            user_sum_dict[row[0]] += row[1]
            user_count_dict[row[0]] += 1

In [31]:
del train
_=gc.collect()

In [32]:
user_bundle_first = pd.read_csv('../input/mydata/archive/archive/user_bundle_first.csv')
user_tag_first = pd.read_csv('../input/mydata/archive/archive/user_tag_first.csv')

In [33]:
user_bundle_first_dict = defaultdict(int)
for cnt,item in enumerate(tqdm(user_bundle_first[['user_id','user_bundle_first']].values)):
    user_bundle_first_dict[item[0]] = item[1]

user_tag_first_dict = defaultdict(int)
for cnt,item in enumerate(tqdm(user_tag_first[['user_id','user_tag_first']].values)):
    user_tag_first_dict[item[0]] = item[1]

In [34]:
def add_first(df,user_bundle_first_dict,user_tag_first_dict):
    temp1 = np.zeros(len(df))
    temp2 = np.zeros(len(df))
    for cnt,row in enumerate(df[['user_id','bundle_id','tags_lsi']].values):
        if row[0] not in user_bundle_first_dict.keys():
            user_bundle_first_dict[row[0]]  = row[1]
            user_tag_first_dict[row[0]]  = row[2]
        temp1[cnt] = user_bundle_first_dict[row[0]]
        temp2[cnt] = user_tag_first_dict[row[0]]
    leature_df = pd.DataFrame({'user_bundle_first':temp1,'user_tag_first':temp2})
    df = pd.concat([df, leature_df], axis=1)
    return df

In [35]:
user_lecture_agg = pd.read_csv('../input/mydata/archive/archive/user_lecture_agg.csv')

In [36]:
user_lecture_sum = defaultdict(int)
user_lecture_count = defaultdict(int)
for cnt,item in enumerate(tqdm(user_lecture_agg[['user_id','sum','count']].values)):
    user_lecture_sum[item[0]] = item[1]
    user_lecture_count[item[0]] = item[2]

In [37]:
hard_question_sum_dict = defaultdict(int)
for cnt,item in enumerate(tqdm(hard_question_sum[['user_id','hard_question_sum']].values)):
    hard_question_sum_dict[item[0]] = item[1]
    
middle_question_sum_dict = defaultdict(int)
for cnt,item in enumerate(tqdm(middle_question_sum[['user_id','middle_question_sum']].values)):
    middle_question_sum_dict[item[0]] = item[1]

easy_question_sum_dict = defaultdict(int)
for cnt,item in enumerate(tqdm(easy_question_sum[['user_id','easy_question_sum']].values)):
    easy_question_sum_dict[item[0]] = item[1]

In [38]:
def add_leature(df,user_lecture_sum,user_lecture_count):
    temp1 = np.zeros(len(df))
    temp2 = np.zeros(len(df))
    for cnt,row in enumerate(df[['user_id','content_type_id']].values):
        user_lecture_count[row[0]] = user_lecture_count[row[0]] + 1
        user_lecture_sum[row[0]] = user_lecture_sum[row[0]] +row[1]
        if row[1] == 0:
            temp1[cnt] = user_lecture_sum[row[0]]
            temp2[cnt] =  user_lecture_sum[row[0]]/user_lecture_count[row[0]]
    leature_df = pd.DataFrame({'user_lecture_sum':temp1,'user_lecture_lv':temp2})
    df = pd.concat([df, leature_df], axis=1)
    return df

In [39]:
question_cnt = pd.read_csv('../input/mydata/archive/archive/easy_question_count.csv')

In [40]:
question_cnt_dict = defaultdict(int)
for cnt,item in enumerate(tqdm(question_cnt[['user_id','easy_question_count']].values)):
    question_cnt_dict[item[0]] = item[1]

In [41]:
def add_leavel(df,hard_question_sum_dict,middle_question_sum_dict,easy_question_sum_dict,question_cnt_dict):
    easy_sum = np.zeros(len(df))
    easy_mean = np.zeros(len(df))
    middle_sum = np.zeros(len(df))
    middle_mean = np.zeros(len(df))
    hard_sum = np.zeros(len(df))
    hard_mean = np.zeros(len(df))
    for cnt,row in enumerate(df[['user_id','answered_correctly_avg_c']].values):
        question_cnt_dict[row[0]] = question_cnt_dict[row[0]] + 1
        if row[1] < 0.6:
            hard_question_sum_dict[row[0]] = hard_question_sum_dict[row[0]] + 1
        if 0.6<=row[1]<0.83:
            middle_question_sum_dict[row[0]] = middle_question_sum_dict[row[0]] + 1
        if row[1] >= 0.83:
            easy_question_sum_dict[row[0]] = easy_question_sum_dict[row[0]] + 1
        easy_sum[cnt] = easy_question_sum_dict[row[0]]
        easy_mean[cnt] = easy_question_sum_dict[row[0]]/question_cnt_dict[row[0]]
        
        middle_sum[cnt] = middle_question_sum_dict[row[0]]
        middle_mean[cnt] = middle_question_sum_dict[row[0]]/question_cnt_dict[row[0]]
        
        hard_sum[cnt] = hard_question_sum_dict[row[0]]
        hard_mean[cnt] = hard_question_sum_dict[row[0]]/question_cnt_dict[row[0]]
    question_lv = pd.DataFrame({'hard_question_sum':hard_sum,'hard_question_mean':hard_mean,'middle_question_sum':middle_sum,'middle_question_mean':middle_mean,'easy_question_sum':easy_sum,'easy_question_mean':easy_mean})
    df = pd.concat([df, question_lv], axis=1)
    return df

In [42]:
env = riiideducation.make_env()
iter_test = env.iter_test()

# SAINT+

In [43]:
MAX_SEQ = 200

In [44]:
class Feed_Forward_block(nn.Module):
    """
    out =  Relu( M_out*w1 + b1) *w2 + b2
    """
    def __init__(self, dim_ff):
        super().__init__()
        self.layer1 = nn.Linear(in_features=dim_ff , out_features=dim_ff)
        self.layer2 = nn.Linear(in_features=dim_ff , out_features=dim_ff)
        #self.dropout = nn.Dropout(0.1)

    def forward(self,ffn_in):
        #return  self.dropout(self.layer2(   F.relu( self.layer1(ffn_in) )   ))
        return  self.layer2(   F.relu( self.layer1(ffn_in) )   )
        

class Encoder_block(nn.Module):
    """
    M = SkipConct(Multihead(LayerNorm(Qin;Kin;Vin)))
    O = SkipConct(FFN(LayerNorm(M)))
    """

    def __init__(self , dim_model, heads_en, total_ex ,total_cat, seq_len):
        super().__init__()
        self.seq_len = seq_len
        self.embd_ex =   nn.Embedding( total_ex , embedding_dim = dim_model )                   # embedings  q,k,v = E = exercise ID embedding, category embedding, and positionembedding.
        self.embd_cat =  nn.Embedding( total_cat, embedding_dim = dim_model )
        self.embd_pos = nn.Embedding(  seq_len , embedding_dim = dim_model )                  #positional embedding
        #self.embd_pr = nn.Embedding(  2 , embedding_dim = dim_model )                  # prior question has explanation embedding

        self.multi_en = nn.MultiheadAttention( embed_dim= dim_model, num_heads= heads_en,  )     # multihead attention    ## todo add dropout, LayerNORM
        self.ffn_en = Feed_Forward_block( dim_model )                                            # feedforward block     ## todo dropout, LayerNorm
        self.layer_norm1 = nn.LayerNorm( dim_model )
        self.layer_norm2 = nn.LayerNorm( dim_model )


    #def forward(self, in_ex, in_cat, in_pr, first_block=True):
    def forward(self, in_ex, in_cat, first_block=True):

        ## todo create a positional encoding ( two options numeric, sine)
        if first_block:
            in_ex = self.embd_ex( in_ex )
            in_cat = self.embd_cat( in_cat )
            #in_pr = self.embd_cat( in_pr )
            #combining the embedings
            #out = in_ex + in_cat + in_pr #+ in_pos                      # (b,n,d)
            out = in_ex + in_cat#+ in_pos                      # (b,n,d)
        else:
            out = in_ex
        
        in_pos = get_pos(self.seq_len)
        in_pos = self.embd_pos( in_pos )
        out = out + in_pos                                      # Applying positional embedding

        out = out.permute(1,0,2)                                # (n,b,d)
        # print('pre multi', out.shape )
        
        #Multihead attention                            
        n,_,_ = out.shape
        out = self.layer_norm1( out )                           # Layer norm
        skip_out = out 
        out, attn_wt = self.multi_en( out , out , out ,
                                attn_mask=get_mask(seq_len=n))  # attention mask upper triangular
        out = out + skip_out                                    # skip connection

        #feed forward
        out = out.permute(1,0,2)                                # (b,n,d)
        out = self.layer_norm2( out )                           # Layer norm 
        skip_out = out
        out = self.ffn_en( out )
        out = out + skip_out                                    # skip connection

        return out


class Decoder_block(nn.Module):
    """
    M1 = SkipConct(Multihead(LayerNorm(Qin;Kin;Vin)))
    M2 = SkipConct(Multihead(LayerNorm(M1;O;O)))
    L = SkipConct(FFN(LayerNorm(M2)))
    """

    def __init__(self,dim_model ,total_in, heads_de, seq_len):
        super().__init__()
        self.seq_len    = seq_len
        self.embd_in    = nn.Embedding(  total_in , embedding_dim = dim_model )                  #interaction embedding
        self.embd_pos   = nn.Embedding(  seq_len , embedding_dim = dim_model )                  #positional embedding
        #self.embd_pr    = nn.Embedding(  2 , embedding_dim = dim_model )
        self.embd_ts    = nn.Embedding(  150, embedding_dim = dim_model )
        self.multi_de1  = nn.MultiheadAttention( embed_dim= dim_model, num_heads= heads_de  )  # M1 multihead for interaction embedding as q k v
        self.multi_de2  = nn.MultiheadAttention( embed_dim= dim_model, num_heads= heads_de  )  # M2 multihead for M1 out, encoder out, encoder out as q k v
        self.ffn_en     = Feed_Forward_block( dim_model )                                         # feed forward layer

        self.layer_norm1 = nn.LayerNorm( dim_model )
        self.layer_norm2 = nn.LayerNorm( dim_model )
        self.layer_norm3 = nn.LayerNorm( dim_model )


    #def forward(self, in_in, en_out, in_pr, ts, first_block=True):
    def forward(self, in_in, en_out, ts, first_block=True):

         ## todo create a positional encoding ( two options numeric, sine)
        if first_block:
            #print(in_in)
            in_in = self.embd_in( in_in )

            #combining the embedings
            out = in_in #+ in_cat #+ in_pos                         # (b,n,d)
        else:
            out = in_in

        in_pos = get_pos(self.seq_len)
        in_pos = self.embd_pos( in_pos )
        out = out + in_pos                                          # Applying positional embedding

        #in_pr = self.embd_pr(in_pr)
        #out = out + in_pr

        #print('ts', ts)
        ts = self.embd_ts(ts)
        out = out + ts
        
        out = out.permute(1,0,2)                                    # (n,b,d)
        # print('pre multi', out.shape )
        n,_,_ = out.shape

        #Multihead attention M1                                     ## todo verify if E to passed as q,k,v
        out = self.layer_norm1( out )
        skip_out = out
        out, attn_wt = self.multi_de1( out , out , out, 
                                     attn_mask=get_mask(seq_len=n)) # attention mask upper triangular
        out = skip_out + out                                        # skip connection

        #Multihead attention M2                                     ## todo verify if E to passed as q,k,v
        en_out = en_out.permute(1,0,2)                              # (b,n,d)-->(n,b,d)
        en_out = self.layer_norm2( en_out )
        skip_out = out
        out, attn_wt = self.multi_de2( out , en_out , en_out,
                                    attn_mask=get_mask(seq_len=n))  # attention mask upper triangular
        out = out + skip_out

        #feed forward
        out = out.permute(1,0,2)                                    # (b,n,d)
        out = self.layer_norm3( out )                               # Layer norm 
        skip_out = out
        out = self.ffn_en( out )                                    
        out = out + skip_out                                        # skip connection

        return out

def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])


def get_mask(seq_len):
    ##todo add this to device
    return torch.from_numpy( np.triu(np.ones((seq_len ,seq_len)), k=1).astype('bool')).to(device) 

def get_pos(seq_len):
    # use sine positional embeddinds
    return torch.arange( seq_len ).unsqueeze(0).to(device) 

class saint(nn.Module):
    def __init__(self,dim_model,num_en, num_de ,heads_en, total_ex ,total_cat,total_in,heads_de,seq_len ):
        super().__init__( )

        self.num_en = num_en
        self.num_de = num_de

        self.encoder = get_clones( Encoder_block(dim_model, heads_en , total_ex ,total_cat,seq_len) , num_en)
        self.decoder = get_clones( Decoder_block(dim_model ,total_in, heads_de,seq_len)             , num_de)

        self.out = nn.Linear(in_features= dim_model , out_features=1)
    
    #def forward(self,in_ex, in_cat, in_pr, in_in, ts):
    def forward(self,in_ex, in_cat, in_in, ts):
        #print(in_ex.shape, in_cat.shape, in_in.shape)

        ## pass through each of the encoder blocks in sequence
        first_block = True
        for x in range(self.num_en):
            if x>=1:
                first_block = False
            #in_ex = self.encoder[x]( in_ex, in_cat , in_pr, first_block=first_block)
            in_ex = self.encoder[x]( in_ex, in_cat, first_block=first_block)
            in_cat = in_ex                                  # passing same output as q,k,v to next encoder block

        
        ## pass through each decoder blocks in sequence
        first_block = True
        for x in range(self.num_de):
            if x>=1:
                first_block = False
            #in_in = self.decoder[x]( in_in , in_ex, in_pr, ts, first_block=first_block )
            in_in = self.decoder[x]( in_in , in_ex, ts, first_block=first_block )

        ## Output layer
        return self.out( in_in ).squeeze(-1)

In [45]:
%%time

skills = joblib.load("/kaggle/input/riiid-sakt-model-dataset-public/skills.pkl.zip")
n_skill = len(skills) + 1
group = joblib.load("/kaggle/input/sakt-models/test_group.pkl.lzma")

CPU times: user 2min 27s, sys: 2.69 s, total: 2min 30s
Wall time: 2min 35s


In [46]:
%%time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

total_cat=7+1
total_in=2*n_skill+1
nn_model = saint(dim_model=256,
            num_en=4,
            num_de=4,
            heads_en=4,
            heads_de=4,
            total_ex=n_skill,
            total_cat=total_cat,
            total_in=total_in,
            seq_len=MAX_SEQ
            )
try:
    nn_model.load_state_dict(torch.load("/kaggle/input/sakt-models/saint_model_200_256_0.785371.pt"))
except:
    nn_model.load_state_dict(torch.load("/kaggle/input/sakt-models/saint_model_200_256_0.785371.pt", map_location='cpu'))
nn_model.to(device)
nn_model.eval()

CPU times: user 1.61 s, sys: 1.17 s, total: 2.78 s
Wall time: 8.12 s


saint(
  (encoder): ModuleList(
    (0): Encoder_block(
      (embd_ex): Embedding(13524, 256)
      (embd_cat): Embedding(8, 256)
      (embd_pos): Embedding(200, 256)
      (multi_en): MultiheadAttention(
        (out_proj): _LinearWithBias(in_features=256, out_features=256, bias=True)
      )
      (ffn_en): Feed_Forward_block(
        (layer1): Linear(in_features=256, out_features=256, bias=True)
        (layer2): Linear(in_features=256, out_features=256, bias=True)
      )
      (layer_norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (layer_norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
    (1): Encoder_block(
      (embd_ex): Embedding(13524, 256)
      (embd_cat): Embedding(8, 256)
      (embd_pos): Embedding(200, 256)
      (multi_en): MultiheadAttention(
        (out_proj): _LinearWithBias(in_features=256, out_features=256, bias=True)
      )
      (ffn_en): Feed_Forward_block(
        (layer1): Linear(in_features=256, out_features=256, 

In [47]:
class TestDataset(Dataset):
    def __init__(self, samples, test_df, skills, max_seq=MAX_SEQ): 
        super(TestDataset, self).__init__()
        self.samples = samples
        self.test_df = test_df
        self.skills = skills
        self.n_skill = len(skills) + 1
        self.max_seq = max_seq
        self.q = np.zeros(self.max_seq, dtype=int)
        self.x = np.zeros(self.max_seq, dtype=int)
        self.qa = np.zeros(self.max_seq, dtype=int)
        self.ts = np.zeros(self.max_seq, dtype=np.int64)
        #self.pr = np.zeros(self.max_seq, dtype=int)
        self.pa = np.zeros(self.max_seq, dtype=int)

    def __len__(self):
        return self.test_df.shape[0]

    def __getitem__(self, index):
        test_info = self.test_df.iloc[index]

        user_id = test_info["user_id"]
        target_id = test_info["content_id"]

        timestamp = test_info["timestamp"]
        #prior = test_info["prior_question_had_explanation"]
        part = test_info['part']

        if user_id in self.samples:
            #q, qa, ts, pr, pa = self.samples[user_id]
            q, qa, ts, pa = self.samples[user_id]

            # concat 
            q = np.append(q, [target_id])
            qa = np.append(qa, [0])
            ts = np.append(ts, [timestamp])
            #pr = np.append(pr, [prior])
            pa = np.append(pa, [part])

            ts[1:] = ts[1:]-ts[:-1]
            ts[0] = 0

            # slice
            q = q[-self.max_seq:]
            qa = qa[-self.max_seq:]
            ts = ts[-self.max_seq:]
            #pr = pr[-self.max_seq:]
            pa = pa[-self.max_seq:]

            # construct interaction
            q += 1
            x = q + qa * self.n_skill
            x[1:] = x[:-1]
            x[0] = 0

            # calculate lag
            ts = ts // 60000
            ts[np.logical_and(ts>=5, ts<10)]=5
            for i in range(1, 144, 1):
                ts[np.where((ts>=i*10) & (ts<(i+1)*10))] = 5+i
            ts[np.where(ts>=1440)] = 149

            # pad
            pad_cnt = len(q) % self.max_seq
            if pad_cnt:
                pad_cnt = self.max_seq - pad_cnt
                padding = np.zeros(pad_cnt)
                q = np.insert(q, 0, padding)
                x = np.insert(x, 0, padding)
                ts = np.insert(ts, 0, padding)
                #pr = np.insert(pr, 0, padding)
                pa = np.insert(pa, 0, padding)
            return x, q, pa, ts
        else:
            self.q[-1] = target_id + 1
            #pr[-1] = prior
            self.pa[-1] = part
            return self.x, self.q, self.pa, self.ts

In [48]:
FEATS = ['prior_question_had_explanation',
 'part',
 'tags_num',
 'community',
 'answered_correctly_avg_c',
 'answered_correctly_std_c',
 'answered_correctly_sum_c',
 'answered_correctly_sum_m',
 'answered_correctly_avg_b',
 'bundle_id',
 'user_correct_cumsum',
 'user_correct_cumcount',
 'user_correctness',
 'attempt',
 'hmean_user_content_accuracy',
 'prior_question_elapsed_time_mean',
 'answered_correctly_avg_p',
 'answered_correctly_sum_p',
 'performance',
 'lag_time',
 'prior_lag_time',
 'explanation_mean',
 'explanation_cumsum',
 'answered_correctly_avg_t',
 'answered_correctly_sum_t',
 'lag_time2','lag_time3','elapsed_time_question',
 'user_lecture_sum',
 'user_lecture_lv', 'hard_question_sum', 'middle_question_sum',
 'easy_question_sum', 'hard_question_mean', 'middle_question_mean',
 'easy_question_mean','tags_lsi','answered_correctly_avg_g','answered_correctly_sum_g',
 'tag_cor_mean','tag_cor_max','tag_cor_min','user_bundle_first','user_tag_first',
 'lag_time_avg_c', 'lag_time_avg_m', 'lag_time_avg_p', 'lag_time_avg_b',
 'lag_time_avg_t', 'lag_time_avg_g']

In [49]:
%%time

model = joblib.load('../input/mydata/lgbm.pkl.lzma')

CPU times: user 24.1 s, sys: 1.52 s, total: 25.6 s
Wall time: 31.1 s


In [50]:
previous_lgbm_test_df = None
previous_nn_test_df = None
nn_question = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv')[['question_id', 'part']]
gc.collect()
print(psutil.virtual_memory().percent)
for (test_df, sample_prediction_df) in iter_test:
    nn_test_df = pd.merge(test_df, nn_question, left_on='content_id', right_on='question_id', how='left')

    if (previous_nn_test_df is not None):
        ans = eval(nn_test_df["prior_group_answers_correct"].iloc[0])
        previous_nn_test_df['answered_correctly'] = ans
        previous_lgbm_test_df['answered_correctly'] = ans

        #lightgbm
        update_user_feats(previous_lgbm_test_df, user_sum_dict,user_count_dict)
        
        #dnn
        previous_nn_test_df = previous_nn_test_df[previous_nn_test_df.content_type_id == False]
        
        prev_group = previous_nn_test_df[['user_id', 'content_id', 'answered_correctly', 'timestamp', 'part']].groupby('user_id').apply(lambda r: (
            r['content_id'].values,
            r['answered_correctly'].values,
            r['timestamp'].values,
            r['part'].values))

        for prev_user_id in prev_group.index:
            if prev_user_id in group:
                group[prev_user_id] = (
                    np.append(group[prev_user_id][0], prev_group[prev_user_id][0])[-MAX_SEQ:], 
                    np.append(group[prev_user_id][1], prev_group[prev_user_id][1])[-MAX_SEQ:], 
                    np.append(group[prev_user_id][2], prev_group[prev_user_id][2])[-MAX_SEQ:], 
                    np.append(group[prev_user_id][3], prev_group[prev_user_id][3])[-MAX_SEQ:], 
                )
 
            else:
                group[prev_user_id] = (
                    prev_group[prev_user_id][0][-MAX_SEQ:], 
                    prev_group[prev_user_id][1][-MAX_SEQ:], 
                    prev_group[prev_user_id][2][-MAX_SEQ:], 
                    prev_group[prev_user_id][3][-MAX_SEQ:], 
                )
            
    previous_nn_test_df = nn_test_df.copy()
    previous_lgbm_test_df = test_df.copy()

    test_df = add_leature(test_df.reset_index(drop=True),user_lecture_sum,user_lecture_count)
    test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop=True)

    ##dnn
    nn_test_df = nn_test_df[test_df.content_type_id == False]
    test_dataset = TestDataset(group, nn_test_df, skills)
    test_dataloader = DataLoader(test_dataset, batch_size=51200, shuffle=False)
    
    outs = []

    for item in test_dataloader:
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()
        pa = item[2].to(device).long()
        ts = item[3].to(device).long()

        with torch.no_grad():
            output = nn_model(target_id, pa, x, ts)
        outs.extend(torch.sigmoid(output)[:, -1].view(-1).data.cpu().numpy())
    
    ##lightgbm
    test_df['prior_question_had_explanation'] = test_df.prior_question_had_explanation.fillna(False).astype('int8')
    test_df['prior_question_elapsed_time_mean'] = test_df.prior_question_elapsed_time.fillna(prior_question_elapsed_time_mean)
    test_df = add_explanation(test_df,explanation_sum_dict,explanation_count_dict)
    test_df = add_user_feats_without_update(test_df, user_sum_dict, user_count_dict)
    
    test_df = pd.merge(test_df, question, left_on='content_id', right_on='question_id', how='left')
    test_df = pd.merge(test_df, answered_correctly_avg_c, on='content_id',  how="left")
    test_df = pd.merge(test_df, answered_correctly_std_c, on='content_id',  how="left")
    test_df = pd.merge(test_df, answered_correctly_sum_c, on='content_id',  how="left")
    test_df = pd.merge(test_df, answered_correctly_sum_m, on='community',  how="left")
    test_df = pd.merge(test_df, answered_correctly_avg_p, on='part',  how="left")
    test_df = pd.merge(test_df, answered_correctly_sum_p, on='part',  how="left")
    test_df = pd.merge(test_df, answered_correctly_avg_b, on='bundle_id',  how="left")
    test_df = pd.merge(test_df, answered_correctly_avg_t, on='task_container_id',  how="left")
    test_df = pd.merge(test_df, answered_correctly_sum_t, on='task_container_id',  how="left")
    test_df = pd.merge(test_df, answered_correctly_avg_g, on='tags_lsi',  how="left")
    test_df = pd.merge(test_df, answered_correctly_sum_g, on='tags_lsi',  how="left")
    
    test_df = pd.merge(test_df, lag_time_avg_b, on='bundle_id',  how="left")
    test_df = pd.merge(test_df, lag_time_avg_c, on='content_id',  how="left")
    test_df = pd.merge(test_df, lag_time_avg_g, on='tags_lsi',  how="left")
    test_df = pd.merge(test_df, lag_time_avg_m, on='community',  how="left")
    test_df = pd.merge(test_df, lag_time_avg_p, on='part',  how="left")
    test_df = pd.merge(test_df, lag_time_avg_t, on='task_container_id',  how="left")

    test_df = add_first(test_df,user_bundle_first_dict,user_tag_first_dict)
    test_df = add_leavel(test_df,hard_question_sum_dict,middle_question_sum_dict,easy_question_sum_dict,question_cnt_dict)
    test_df['hmean_user_content_accuracy'] = 2 * (
        (test_df['user_correctness'] * test_df['answered_correctly_avg_c']) /
        (test_df['user_correctness'] + test_df['answered_correctly_avg_c'])
    )
    
    test_df['performance'] = test_df['answered_correctly_avg_c'] - test_df['user_correctness']
    test_df['timestamp'] = test_df['timestamp']/(1000*3600)
    test_df = add_time(test_df, time_last_dict,time_last_dict2,time_last_dict3)
    
    test_df['prior_question_elapsed_time_mean'] = test_df['prior_question_elapsed_time_mean']/(1000*3600)
    test_df = add_prior_time(test_df,prior_time_last_dict)
    test_df['elapsed_time_question'] = test_df['lag_time'] - test_df['prior_question_elapsed_time_mean'] 
    test_df["attempt"] = test_df[["user_id", "content_id"]].apply(lambda row: get_max_attempt(row["user_id"], row["content_id"]), axis=1)
    
    ##blending
    test_df['answered_correctly'] = 0.5*model.predict(test_df[FEATS].values) + 0.5*np.array(outs)

    env.predict(test_df[['row_id', 'answered_correctly']])
    
    _ = gc.collect()

46.5
